In [ ]:
# To start running this notebook: in a shell, go to the home directory of the github repo.
#   cd src
#   conda create geocache python=3.11
#   conda activate geocache
#   conda install numpy jupyter
#   jupyter notebook usd-separate.ipynb
# Then you can shift-enter to run each cell here.

!pip install usd-core

In [ ]:
# Test compressing the biggest animation using PCA with very stupid clustering.

In [ ]:
infile = "/Users/bhudson/projects/ALab/ALab/entry.usda"
outdir = "/Users/bhudson/projects/geocache-compression/build"

In [ ]:
from pxr import Usd
from embedding.io import separate_usd
from embedding.io import create_embedding
from embedding.io import read_embedding
from embedding.metric import Report
from embedding.io import read_binfile
import os

stage = Usd.Stage.Open(infile)
mpu = stage.GetMetadata("metersPerUnit")
print(mpu)

In [ ]:
package = separate_usd(infile, outdir, verbose=False)
print(f"{package.inputfile} has {len(package.headers)} animations")

In [ ]:
# Test just doing one embedding.
# header = max(package.headers, key=lambda h: h.numbytes()) # compress the biggest file
header = package.get_header("/root/remi/head_M_hrc/GEO/head_M_hrc/facePlate_M_geo")
print(f"{header}")
files = create_embedding(header, clustersize=1000, verbose=True)
print(files)

In [ ]:
postdata = read_embedding(header, files, verbose=True)
predata = read_binfile(header)
report = Report(predata, postdata, sum(os.path.getsize(path) for path in files))
print(f"{header.path}")
report.print_report(mpu)

In [ ]:
# Test *all* the animations.
def run_report(header):
    files = create_embedding(header, clustersize=1000)
    size = sum(os.path.getsize(path) for path in files)
    predata = read_binfile(header)
    postdata = read_embedding(header, files)
    r = Report(predata, postdata, size)
    compression_ratio = 1 - r.compressed_size / r.original_size
    print(f"{compression_ratio: 5.2%} | {r.original_size} | {header.path}")
    return r


reports = {header.path: run_report(header) for header in package.headers}

In [ ]:
orig = sum(r.original_size for r in reports.values())
compressed = sum(r.compressed_size for r in reports.values())
print(f"{orig} vs {compressed} -- {1-compressed/orig:.2%} compression")

h = max(r.hausdorff for r in reports.values()) * mpu
linf = max(r.Linf for r in reports.values()) * mpu
print(f"Max error: {h} m distance, {linf} m any single coordinate")

corrected = max(r.corrected_Linf for r in reports.values()) * mpu
uncorrected = sum(r.numuncorrectable for r in reports.values())
numvalues = sum(r.original_numvalues for r in reports.values())
print(f"{uncorrected} uncorrectable out of {numvalues} ({uncorrected/numvalues:%})")